In [1]:
import argparse
import json
import os
# import torch
from pathlib import Path
from tqdm import tqdm

# data_abs_dir = Path(__file__).parent / "data"
data_abs_dir = "E:\A25cun\coder\deepcoder\evaluation\HumanEval\data"


In [2]:
import sys
sys.path.append('E:/A25cun/coder')
import os
from openai import OpenAI
from swarm import Swarm, Agent
from deepcoder.utils import pretty_print_messages,process_and_print_streaming_response,extract_generation_code, languge_settings
import sys

api_key = "sk-1fac7836ded54cfabf056520607a4c4d"
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_BASE_URL"] = "https://api.deepseek.com"
openai_client = OpenAI(api_key="sk-1fac7836ded54cfabf056520607a4c4d", base_url="https://api.deepseek.com")
client = Swarm(openai_client)

AGENT_CODER_PROMPT = """
**Role**: You are a software programmer.

**Task**: As a programmer, you are required to complete the function. Use a Chain-of-Thought approach to break down the problem, create pseudocode, and then write the code in Python language.

**Instructions**: 
1. **Understand and Clarify**: Make sure you understand the task. 
2. **Algorithm/Method Selection**: Decide on the most efficient way. 
3. **Pseudocode Creation**: Write down the steps you will follow in pseudocode. 
4. **Code Generation**: Translate your pseudocode into executable Python code.
NOTE: Generate usable code by this Chain-of-Thought approach without summarizing or providing test cases.

**Code Formatting**: Please write code in 
```python
[Code]
``` 
format.

"""

AGENT_TESTER_PROMPT = """
**Role**: As a tester, your task is to create comprehensive test cases for the incomplete function. 

**Instructions**: 
1. Implement a comprehensive set of test cases following the guidelines above. 
2. Ensure each test case is well-documented with comments explaining the scenario it covers. 
3. Pay special attention to edge cases as they often reveal hidden bugs. 
4. For large-scale tests, focus on the function's efficiency and performance under heavy loads.
NOTE: Generate only the test code section, without creating the specific functions or providing a summary.

- The format of test cases should be:
```python
assert function_name(input) == expected_output, "Test Case Description"
```
"""

AGENT_DEBUG_PROMPT= """
NOTICE
1. Role: You are a Development Engineer or QA engineer;
2. Task: You received this message from another Development Engineer or QA engineer who ran or tested your code. 
Based on the message, first, figure out your own role, i.e. Engineer or QaEngineer,
then rewrite the development code or the test code based on your role, the error, and the summary, such that all bugs are fixed and the code performs well.
Attention: Use '##' to split sections, not '#', and '## <SECTION_NAME>' SHOULD WRITE BEFORE the test case or script and triple quotes.
"""

CODER_PROMPT_TEMPLATE = """
The message is as follows:
# Requirement

{requirement}

---
Now you should start writing the code:
## Write code with triple quote. Do your best to implement THIS IN ONLY ONE FILE.
"""

TESTER_PROMPT_TEMPLATE = """
The message is as follows:
# Requirement

{requirement}

---
Now you should start writing test case for the code:
## Write code with triple quote. Do your best to implement THIS IN ONLY ONE FILE.
"""

DEBUG_PROMPT_TEMPLATE = """
The message is as follows:
# Legacy Code
```python
{code}
```
---
# Unit Test Code
```python
{test_code}
```
---
# Console logs
```text
{logs}
```
 the code to rewrite: Write code with triple quote. Do your best to implement THIS IN ONLY ONE FILE.
---
Now you should start rewriting the code:
"""

coder_agent = Agent(
    name="Code Generator agent",
    model="deepseek-coder",
    instructions=AGENT_CODER_PROMPT,
    functions=[]
)

tester_agent = Agent(
    name="Tester agent",
    model="deepseek-chat",
    instructions=AGENT_TESTER_PROMPT,
    functions=[]
)

debug_agent = Agent(
    name="Runner agent",
    model="deepseek-chat",
    instructions=AGENT_DEBUG_PROMPT,
    functions=[]
)



In [3]:
# messages = [{"role":"user","content":"hello"}]
# response = client.run(
#         agent=coder_agent,
#         messages=messages,
#         stream=True
#     )

# process_and_print_streaming_response(response)

In [4]:
def build_deepseekcoder_instruction(languge: str, question: str):
    return '''
Please continue to complete the function. You are not allowed to modify the given code and do the completion only. Please return all completed function in a codeblock. Here is the given code to do completion:
```{}
{}
```
'''.strip().format(languge.lower(), question.strip())

In [5]:
from deepcoder.utils import extract_generation_code
def generate_one(example, lang):
    prompt = build_deepseekcoder_instruction(languge_settings[lang]['full_name'], example['prompt'])
    # response = client.run(
    #     agent=coder_agent,
    #     messages= [{'role': 'user', 'content': prompt }],
    #     stream=False
    # )
    # example['output'] = response.messages[-1]["content"]
    example['output'] = "```python\nprint(\"hello\")\n```"
    return extract_generation_code(example, lang_code=lang)

In [6]:
def generate_main(args):
    lang = args.language
    saved_path = args.output_path
    temp_dir = args.temp_dir
    os.makedirs(temp_dir, exist_ok=True)
    problem_file = os.path.join(data_abs_dir, f"humaneval-{lang}.jsonl")

    examples = [json.loads(x) for x in open(problem_file) if x.strip()]
    print("Read {} examples for evaluation over.".format(len(examples)))

    generated_examples = []
    for ex in tqdm(examples, desc='Generating'):
        gen_example = generate_one(ex, args.language)
        generated_examples.append(gen_example)

    print("Generate all over!!!")
    with open(saved_path, 'w', encoding='utf-8') as fw:
        for ex in generated_examples:
            fw.write(json.dumps(ex) + '\n')
        print("Save {} processed examples into {} over!".format(len(generated_examples), saved_path))

    return saved_path,temp_dir,problem_file
    # result = evaluate_functional_correctness(
    #     input_file=saved_path,
    #     tmp_dir=temp_dir,
    #     n_workers=8,
    #     timeout=3.0,
    #     problem_file=problem_file,
    #     language=lang
    # )
    # print(lang, result, model_name_or_path)
    # pass



In [7]:
def evaluation_only(args):
    lang = args.language
    temp_dir = args.temp_dir
    assert os.path.exists(args.output_path), "Not fond output file: {}".format(args.output_path)
    os.makedirs(temp_dir, exist_ok=True)

    output_name = os.path.basename(args.output_path)
    print("- output name : ",output_name)
    output_examples = [json.loads(x) for x in open(args.output_path) if x.strip()]
    print("output example",output_examples[0])
    processed_examples = [extract_generation_code(ex, lang) for ex in tqdm(output_examples, "Processing")]
    processed_path = os.path.join(temp_dir, output_name)
    
    with open(processed_path, 'w', encoding='utf-8') as fw:
        for ex in processed_examples:
            fw.write(json.dumps(ex) + '\n')
        print("Save {} processed examples into {} over!".format(len(processed_examples), processed_path))

    problem_file = os.path.join(data_abs_dir, f"humaneval-{lang}.jsonl")
    from human_eval.evaluation import evaluate_functional_correctness
    print("processed_path:",processed_path)
    print("tmp_dir : ",temp_dir)
    print("problem_file : ",problem_file)
    result = evaluate_functional_correctness(
        input_file=processed_path,
        tmp_dir=temp_dir,
        n_workers=8,
        timeout=3.0,
        problem_file=problem_file,
        language=lang
    )
    print(lang, result)

In [8]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--output_path', type=str, help="output path of your generation",default="output/python.jsonl")
    parser.add_argument('--language', type=str, help="langauge",default="python")
    parser.add_argument('--temp_dir', type=str, help="temp dir for evaluation", default="tmp")
    import sys  
    sys.argv = ['your_script.py',   
                '--output_path', 'E:\A25cun\coder\deepcoder\evaluation\output\python.jsonl',   
                '--language', 'python',   
                '--temp_dir', 'tmp']  
    args = parser.parse_args()
    # generate_main(args)
    # evaluation_only(args)
    pass

In [9]:
from human_eval.evaluation import evaluate_functional_correctness
result = evaluate_functional_correctness(
    input_file="tmp\python.jsonl",
    tmp_dir="tmp",
    # n_workers=8,
    timeout=3.0,
    problem_file="HumanEval\data\humaneval-python.jsonl",
    language="python"
)
print(result)

e:\miniconda\envs\coder\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Reading samples...


100%|██████████| 164/164 [00:00<00:00, 9394.89it/s]


Running test suites...


  0%|          | 0/164 [00:00<?, ?it/s]

...

result: {'task_id': 'Python/157', 'completion_id': 0, 'result': "failed: module 'signal' has no attribute 'setitimer'", 'passed': False, 'finish': -1, 'code': 'import math\nimport re\nimport sys\nimport copy\nimport datetime\nimport itertools\nimport collections\nimport heapq\nimport functools\nimport hashlib\nimport numpy\nimport numpy as np\nimport string\nfrom typing import *\nfrom collections import *\n\nprint("hello")\n\n\ndef check(right_angle_triangle):\n\n    # Check some simple cases\n    assert right_angle_triangle(3, 4, 5) == True, "This prints if this assert fails 1 (good for debugging!)"\n    assert right_angle_triangle(1, 2, 3) == False\n    assert right_angle_triangle(10, 6, 8) == True\n    assert right_angle_triangle(2, 2, 2) == False\n    assert right_angle_triangle(7, 24, 25) == True\n    assert right_angle_triangle(10, 5, 7) == False\n    assert right_angle_triangle(5, 12, 13) == True\n    assert right_angle_triangle(15, 8, 17) == True\n    assert right_angle_triangl

 94%|█████████▍| 154/164 [00:11<00:00, 18.29it/s]

result: {'task_id': 'Python/158', 'completion_id': 0, 'result': "failed: module 'signal' has no attribute 'setitimer'", 'passed': False, 'finish': -1, 'code': 'import math\nimport re\nimport sys\nimport copy\nimport datetime\nimport itertools\nimport collections\nimport heapq\nimport functools\nimport hashlib\nimport numpy\nimport numpy as np\nimport string\nfrom typing import *\nfrom collections import *\n\nprint("hello")\n\n\ndef check(find_max):\n\n    # Check some simple cases\n    assert (find_max(["name", "of", "string"]) == "string"), "t1"\n    assert (find_max(["name", "enam", "game"]) == "enam"), \'t2\'\n    assert (find_max(["aaaaaaa", "bb", "cc"]) == "aaaaaaa"), \'t3\'\n    assert (find_max(["abc", "cba"]) == "abc"), \'t4\'\n    assert (find_max(["play", "this", "game", "of","footbott"]) == "footbott"), \'t5\'\n    assert (find_max(["we", "are", "gonna", "rock"]) == "gonna"), \'t6\'\n    assert (find_max(["we", "are", "a", "mad", "nation"]) == "nation"), \'t7\'\n    assert (

100%|██████████| 164/164 [00:11<00:00, 14.22it/s]


{'pass@1': np.float64(0.0)}
{'pass@1': np.float64(0.0)}


In [10]:
generate_one({"prompt":"写一个打印hello world的代码","task_id":"2"},"python")

Failed to extract code block with error `list index out of range`:
>>> Task: 2
>>> Output:
```python
print("hello")
```


{'prompt': '写一个打印hello world的代码',
 'task_id': '2',
 'output': '```python\nprint("hello")\n```',
 'generation': '写一个打印hello world的代码\n```python\nprint("hello")\n```'}